# PANDAS FOUNDATIONS-II

In [ ]:
# Load Packages
import numpy as np
import pandas as pd

### Creating a DataFrame from Excel File

In [ ]:
df=pd.read_excel("data/Stocks.xlsx",index_col='Date')
df

### Inspecting your DataFrame

In [ ]:
df.info()

### BROADCASTING

In [ ]:
df.iloc[::3,-1]=np.nan
df

In [ ]:
df.info()

### Creating a Series Object from DataFrame

In [ ]:
low=df['Low']
low

In [ ]:
type(low)

#### Creating One Dimensional DataFrame

In [ ]:
low_df=df[['Low']]
low_df

### Creating an ndarray from Series

In [ ]:
lows=low.values

In [ ]:
type(lows)

In [ ]:
lows.shape

### Creating an ndarray from DataFrame

In [ ]:
np_log10=np.log10(df.values)
np_log10.shape

In [ ]:
np_log10

In [ ]:
type(np_log10)

### Zip Lists to build a DataFrame

In [ ]:
list_keys=['Country', 'Total']
list_values=[['United States', 'Soviet Union', 'United Kingdom'], [1118, 473, 273]]

data=dict(zip(list_keys,list_values))
print(data)
df=pd.DataFrame(data)
df

### Labeling your data

In [ ]:
# Build a list of labels: list_labels
list_labels = ['Country','Total Pop']

# Assign the list of labels to the columns attribute: df.columns
df.columns = list_labels
df

### Building DataFrames using broadcasting

In [ ]:
cities=['Manheim',
 'Preston park',
 'Biglerville',
 'Indiana',
 'Curwensville',
 'Crown',
 'Harveys lake',
 'Mineral springs',
 'Cassville',
 'Hannastown',
 'Saltsburg',
 'Tunkhannock',
 'Pittsburgh',
 'Lemasters',
 'Great bend']

# Make a string with the value 'PA': state
state = 'PA'

# Construct a dictionary: data
data = {'state':state, 'city':cities}

# Construct a DataFrame from dictionary data: df
df = pd.DataFrame(data)

# Print the DataFrame
df

### Building DataFrame from CSV

In [ ]:
filepath='data/SN_d_tot_V2.0.csv'
col_names=['year','month','day','sunspots','definite']
sunspots=pd.read_csv(filepath,delimiter=";",header=None,usecols=[0,1,2,4,7],names=col_names,
                     na_values={'sunspots':['  -1']},parse_dates=[[0,1,2]],index_col='year_month_day')
sunspots.head()

### Saving DataFrame as CSV

In [ ]:
out='out.csv'
sunspots.to_csv(out)

## Handling Missing Values

##### NONE - Using Pythonic way to represent Missing

In [ ]:
vals1 = np.array([1, None, 3, 4])
vals1

This ``dtype=object`` means that the best common type representation NumPy could infer for the contents of the array is that they are Python objects.
While this kind of object array is useful for some purposes, any operations on the data will be done at the Python level, with much more overhead than the typically fast operations seen for arrays with native types:

In [ ]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

The use of Python objects in an array also means that if you perform aggregations like ``sum()`` or ``min()`` across an array with a ``None`` value, you will generally get an error:

In [ ]:
vals1.sum()

### ``NaN``: Missing numerical data

The other missing data representation, ``NaN`` (acronym for *Not a Number*), is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [ ]:
vals2 = np.array([1, np.nan, 3, 4]) 
print(vals2)
vals2.dtype

Notice that NumPy chose a native floating-point type for this array: this means that unlike the object array from before, this array supports fast operations pushed into compiled code.
You should be aware that ``NaN`` is a bit like a data virus–it infects any other object it touches.
Regardless of the operation, the result of arithmetic with ``NaN`` will be another ``NaN``:

In [ ]:
1 + np.nan

Note that this means that aggregates over the values are well defined (i.e., they don't result in an error) but not always useful:

In [ ]:
vals2.sum(), vals2.min(), vals2.max()

NumPy does provide some special aggregations that will ignore these missing values:

In [ ]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

### NaN and None in Pandas

``NaN`` and ``None`` both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [ ]:
pd.Series([1, np.nan, 2, None])

For types that don't have an available sentinel value, Pandas automatically type-casts when NA values are present.
For example, if we set a value in an integer array to ``np.nan``, it will automatically be upcast to a floating-point type to accommodate the NA:

In [ ]:
x = pd.Series(range(2), dtype=int)
x

In [ ]:
x[0] = None
x

## Operating on Null Values

As we have seen, Pandas treats ``None`` and ``NaN`` as essentially interchangeable for indicating missing or null values.
To facilitate this convention, there are several useful methods for detecting, removing, and replacing null values in Pandas data structures.
They are:

- ``isnull()``: Generate a boolean mask indicating missing values
- ``notnull()``: Opposite of ``isnull()``
- ``dropna()``: Return a filtered version of the data
- ``fillna()``: Return a copy of the data with missing values filled or imputed

We will conclude this section with a brief exploration and demonstration of these routines.

### Detecting null values
Pandas data structures have two useful methods for detecting null data: ``isnull()`` and ``notnull()``.
Either one will return a Boolean mask over the data. For example:

In [ ]:
data = pd.Series([1, np.nan, 'hello', None])
data

In [ ]:
data.isnull()

Boolean masks can be used directly as a ``Series`` or ``DataFrame`` index:

In [ ]:
data[data.notnull()]

The ``isnull()`` and ``notnull()`` methods produce similar Boolean results for ``DataFrame``s.

### Dropping null values

In addition to the masking used before, there are the convenience methods, ``dropna()``
(which removes NA values) and ``fillna()`` (which fills in NA values). For a ``Series``,
the result is straightforward:

In [ ]:
data.dropna()

For a ``DataFrame``, there are more options.
Consider the following ``DataFrame``:

In [ ]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

We cannot drop single values from a ``DataFrame``; we can only drop full rows or full columns.
Depending on the application, you might want one or the other, so ``dropna()`` gives a number of options for a ``DataFrame``.

By default, ``dropna()`` will drop all rows in which *any* null value is present:

In [ ]:
df.dropna()

In [ ]:
df.dropna?

Alternatively, you can drop NA values along a different axis; ``axis=1`` drops all columns containing a null value:

In [ ]:
df.dropna(axis=1)

But this drops some good data as well; you might rather be interested in dropping rows or columns with *all* NA values, or a majority of NA values.
This can be specified through the ``how`` or ``thresh`` parameters, which allow fine control of the number of nulls to allow through.

The default is ``how='any'``, such that any row or column (depending on the ``axis`` keyword) containing a null value will be dropped.
You can also specify ``how='all'``, which will only drop rows/columns that are *all* null values:

In [ ]:
df[3] = np.nan
df

In [ ]:
df.dropna(axis='columns', how='all', inplace=True)
df

For finer-grained control, the ``thresh`` parameter lets you specify a minimum number of non-null values for the row/column to be kept:

In [ ]:
df.dropna(axis='rows', thresh=3)

Here the first and last row have been dropped, because they contain only two non-null values.

### Filling null values

Sometimes rather than dropping NA values, you'd rather replace them with a valid value.
This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values.
You could do this in-place using the ``isnull()`` method as a mask, but because it is such a common operation Pandas provides the ``fillna()`` method, which returns a copy of the array with the null values replaced.

Consider the following ``Series``:

In [ ]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

In [ ]:
data.fillna(0)

We can specify a forward-fill to propagate the previous value forward:

In [ ]:
# forward-fill
data.fillna(method='ffill')

In [ ]:
# back-fill
data.fillna(method='bfill')

For ``DataFrame``s, the options are similar, but we can also specify an ``axis`` along which the fills take place:

In [ ]:
df

In [ ]:
df.fillna(method='ffill', axis=1)

Notice that if a previous value is not available during a forward fill, the NA value remains.

## Dates and Time in Python

### Native Python dates and times: ``datetime`` and ``dateutil``

Python's basic objects for working with dates and times reside in the built-in ``datetime`` module.
Along with the third-party ``dateutil`` module, you can use it to quickly perform a host of useful functionalities on dates and times.
For example, you can manually build a date using the ``datetime`` type:

In [ ]:
from datetime import datetime
datetime(year=2017, month=4, day=10)

In [ ]:
datetime.today()

Or, using the ``dateutil`` module, you can parse dates from a variety of string formats:

In [ ]:
from dateutil import parser
date = parser.parse("15th of August, 2015")
date

Once you have a ``datetime`` object, you can do things like printing the day of the week:

In [ ]:
print ("Day of the week:", date.strftime('%A'))
print ("Week of the year:", date.strftime('%W'))


In [ ]:
date + 5

### Typed arrays of times: NumPy's ``datetime64``

The weaknesses of Python's datetime format inspired the NumPy team to add a set of native time series data type to NumPy.
The ``datetime64`` dtype encodes dates as 64-bit integers, and thus allows arrays of dates to be represented very compactly.
The ``datetime64`` requires a very specific input format:

In [ ]:
import numpy as np
date = np.array('2015-07-04', dtype=np.datetime64)
date

Once we have this date formatted, however, we can quickly do vectorized operations on it:

In [ ]:
date + np.arange(12)

Here is a minute-based datetime:

In [ ]:
np.datetime64('2015-07-04 12:00')

Notice that the time zone is automatically set to the local time on the computer executing the code.
You can force any desired fundamental unit using one of many format codes; for example, here we'll force a nanosecond-based time:m

In [ ]:
np.datetime64('2015-07-04 12:59:59.50', 'ns')

### Dates and times in pandas: best of both worlds

Pandas builds upon all the tools just discussed to provide a ``Timestamp`` object, which combines the ease-of-use of ``datetime`` and ``dateutil`` with the efficient storage and vectorized interface of ``numpy.datetime64``.
From a group of these ``Timestamp`` objects, Pandas can construct a ``DatetimeIndex`` that can be used to index data in a ``Series`` or ``DataFrame``; we'll see many examples of this below.

For example, we can use Pandas tools to repeat the demonstration from above.
We can parse a flexibly formatted string date, and use format codes to output the day of the week:

In [ ]:
import pandas as pd
date = pd.to_datetime("15th of August, 2017")
date

In [ ]:
date.strftime('%A')

Additionally, we can do NumPy-style vectorized operations directly on this same object:

In [ ]:
date + pd.to_timedelta(np.arange(12), 'D')

# Indexing pandas time series

Dates are parsed in ISO 8601 YYYY-MM-DD HH:MM:SS

In [ ]:
sunspots.info()

In [ ]:
sunspots.head()

### Flexibility with Datetime Index

In [ ]:
sunspots.loc['February 5,1818']

In [ ]:
sunspots.loc['Feb-5-1818']

In [ ]:
sunspots['1818']

In [ ]:
sunspots.loc['1818-02']

In [ ]:
#Slicing Dates/Times
sunspots.loc['1818-01':'1818-06']

In [ ]:
dat=pd.to_datetime('2015-01-01 091234')
print(dat)
type(dat)

<!--NAVIGATION-->
< [Pandas Foundations-I](04_Pandas_Foundations_I.ipynb) | [Pandas Advanced-I](06_Pandas_Advanced_I.ipynb) >